In [ ]:
import torch
from unet.eff_unet_mod import Eff_Unet  # 根据你的项目结构调整导入路径

# from unet import Eff_Unet
model = Eff_Unet(
    layers=[5, 5, 15, 10],
    embed_dims=[40, 80, 192, 384],
    downsamples=[True, True, True, True],
    vit_num=6,
    drop_path_rate=0.1,
    num_classes=8)
# 2. 构造一个与训练时相同尺寸的假输入
dummy_input = torch.randn(1, 3, 224, 224)
# 1. 实例化模型（请填入与你训练时一致的超参数）
# layers      = [1, 1, 1, 1]
# embed_dims  = [64, 128, 256, 512]
# downsamples = [True, True, True, True]
# model       = Eff_Unet(
#     layers=layers,
#     embed_dims=embed_dims,
#     downsamples=downsamples,
#     num_classes=9      # 或者你实际的类别数
# )
# model.eval()



# 3. 导出为 ONNX
onnx_path = "eff_unet2.onnx"
torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    export_params=True,
    opset_version=14,
    do_constant_folding=True,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
        "input": {0: "batch_size"},
        "output": {0: "batch_size"},
    },

)
print(f"✅ ONNX 模型已保存到 {onnx_path}")

# 4. （可选）加载并查看 ONNX 计算图结构
try:
    import onnx
    from onnx import helper

    onnx_model = onnx.load(onnx_path)
    onnx.checker.check_model(onnx_model)
    # 打印前 1000 字符，必要时可调整或直接打印整个 graph
    print(helper.printable_graph(onnx_model.graph)[:1000] + "\n...")
except ImportError:
    print("❗ 未安装 onnx 库，请先运行：pip install onnx")


/root/miniconda3/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/root/miniconda3/lib/python3.10/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


UnsupportedOperatorError: Exporting the operator 'aten::scaled_dot_product_attention' to ONNX opset version 11 is not supported. Support for this operator was added in version 14, try exporting with this version.

In [ ]:
import onnx
from onnx.tools.net_drawer import GetPydotGraph

# 1) Load your ONNX model
model = onnx.load("eff_unet.onnx")

# 2) Build a pydot graph, supplying a small lambda to label each node
graph = GetPydotGraph(
    model.graph,
    name=model.graph.name,
    rankdir="TB",
    node_producer=lambda node: node.name or node.output[0]  # use node.name if set, else its first output
)

# 3) Dump out as SVG
graph.write_svg("eff_unet.svg")
print("✅ Wrote eff_unet.svg")


ImportError: cannot import name 'NodeIdent' from 'onnx.tools.net_drawer' (/root/miniconda3/lib/python3.10/site-packages/onnx/tools/net_drawer.py)

In [1]:
# 1. 安装依赖
# pip install hiddenlayer graphviz torch

import torch
import hiddenlayer as hl
from unet.eff_unet_mod import Eff_Unet  # 根据你的项目结构调整导入路径

# from unet import Eff_Unet
model = Eff_Unet(
    layers=[5, 5, 15, 10],
    embed_dims=[40, 80, 192, 384],
    downsamples=[True, True, True, True],
    vit_num=6,
    drop_path_rate=0.1,
    num_classes=8)
# 1. 实例化模型（请填入与你训练时一致的超参数）
# layers      = [1, 1, 1, 1]
# embed_dims  = [64, 128, 256, 512]
# downsamples = [True, True, True, True]
# model       = Eff_Unet(
#     layers=layers,
#     embed_dims=embed_dims,
#     downsamples=downsamples,
#     num_classes=9      # 或者你实际的类别数
# )
# model.eval()

# 2. 构造一个与训练时相同尺寸的假输入
dummy_input = torch.randn(1, 3, 224, 224)
model.eval()

# 3. 生成结构图
# dummy_input = torch.zeros(1,3,224,224)
graph = hl.build_graph(model, dummy_input)  
graph.theme = hl.graph.THEMES["blue"]       # 选择主题，或改成你喜欢的配色
graph.save("model_structure", format="png") # 输出 PNG 文件

/root/miniconda3/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/root/miniconda3/lib/python3.10/site-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


AttributeError: module 'torch.onnx' has no attribute '_optimize_trace'

In [3]:
# 安装依赖
# pip install torch torchviz graphviz

import torch
import torch.fx as fx
from torchviz import make_dot
from unet.eff_unet_mod import Eff_Unet  # 替换为你的模块路径

# 1. 实例化模型
model = Eff_Unet(
    layers=[5, 5, 15, 10],
    embed_dims=[40, 80, 192, 384],
    downsamples=[True, True, True, True],
    vit_num=6,
    drop_path_rate=0.1,
    num_classes=8,
    resolution=224,
)
model.eval()

# 2. 用 FX 追踪得到 GraphModule
dummy_input = torch.zeros(1, 3, 224, 224)
traced: fx.GraphModule = fx.symbolic_trace(model)

# 3. 将 FX Graph 转为计算图，然后渲染
#    make_dot 会显示节点名为 fx 节点（call_module）对应的子模块
y = traced(dummy_input)
dot = make_dot(y, params=dict(model.named_parameters()))
dot.format = 'svg'
dot.render('eff_unet_fx_graph')

print("✅ 已生成 eff_unet_fx_graph.svg，可查看子模块和数据流。")

✅ 已生成 eff_unet_fx_graph.svg，可查看子模块和数据流。
